In [1]:
import torch
import segmentation_models_pytorch as smp
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from torchvision import transforms
from PIL import Image
import os
import glob
import re
import imageio.v2 as imageio  # Explicitly use version 2 API
import rasterio
from rasterio.transform import from_origin
from rasterio.crs import CRS
from torch.utils.data import get_worker_info
from sklearn.preprocessing import MinMaxScaler
import torch.utils.data as data
from torch.amp import autocast, GradScaler


ModuleNotFoundError: No module named 'segmentation_models_pytorch'

In [ ]:

# Inputs
transaction_ID = '200008'

# Set device (use GPU if available, otherwise fallback to CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define U-Net model
ENCODER = 'efficientnet-b7'
CLASSES = ['solar_panel']
ACTIVATION = 'sigmoid'

model = smp.Unet(
    in_channels=4,  # 4 for all bands
    encoder_name=ENCODER, 
    encoder_weights=None,  # No pretraining, since we are loading trained weights
    classes=len(CLASSES), 
    activation=ACTIVATION,
)

# Move model to device
model = model.to(device)

# Load trained weights
weights_path = os.path.join(os.path.expanduser("~"), "satellite-ml-solarp-detection", "models", "weights", "u-net_efficientnet-b7_v1", "unet-seed23_weights.pth")

if os.path.exists(weights_path):
    model.load_state_dict(torch.load(weights_path, map_location=device))
    print("Model weights loaded successfully.")
else:
    print("Error: Weights file not found!")

# Set model to evaluation mode
model.eval()

# Function to extract the numeric value from filenames
def numeric_sort_key(filepath):
    match = re.search(r'\d+', filepath)
    return int(match.group()) if match else 0

# Get all the input images and sort them numerically
folder_data_input = sorted(
    glob.glob(os.path.join(os.path.expanduser("~"), "satellite-ml-solarp-detection", "acquisition", transaction_ID, "*tif")),
    key=numeric_sort_key
)

input_image_paths = folder_data_input[:]

# Get all predicted images
folder_predicted_images = sorted(
    glob.glob(os.path.join(os.path.expanduser("~"), "satellite-ml-solarp-detection", "predictions", transaction_ID, "*tif")),
    key=numeric_sort_key
)

predicted_image_paths = folder_predicted_images[:]

# Generate Report
fig, axes = plt.subplots(2, len(input_image_paths), figsize=(20, 10))

for idx, (input_path, pred_path) in enumerate(zip(input_image_paths, predicted_image_paths)):
    input_img = imageio.imread(input_path)
    pred_img = imageio.imread(pred_path)
    
    axes[0, idx].imshow(input_img)
    axes[0, idx].set_title(f"Input {idx+1}")
    axes[0, idx].axis("off")
    
    axes[1, idx].imshow(pred_img, cmap="gray")
    axes[1, idx].set_title(f"Prediction {idx+1}")
    axes[1, idx].axis("off")

plt.tight_layout()
plt.show()

# Compute Statistics
def compute_statistics(predicted_paths):
    stats = []
    for path in predicted_paths:
        pred_img = imageio.imread(path)
        total_pixels = pred_img.size
        positive_pixels = np.sum(pred_img > 0)
        coverage_percentage = (positive_pixels / total_pixels) * 100
        stats.append({
            "File": os.path.basename(path),
            "Total Pixels": total_pixels,
            "Positive Pixels": positive_pixels,
            "Coverage %": coverage_percentage
        })
    return pd.DataFrame(stats)

stats_df = compute_statistics(predicted_image_paths)
print(stats_df)

# Save the report as a CSV
report_path = os.path.join(os.path.expanduser("~"), "satellite-ml-solarp-detection", "reports", f"report_{transaction_ID}.csv")
os.makedirs(os.path.dirname(report_path), exist_ok=True)
stats_df.to_csv(report_path, index=False)
print(f"Report saved: {report_path}")
